In [1]:
import numpy as np
import torch
import torchvision.io as io
import torch.utils.data as data
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [3]:
df = pd.read_csv(r"C:\Users\Micha\TMNIST\TMNIST_Data.csv")
X = df.drop(columns = {'names', 'labels'})/255
X = X.values.reshape(X.shape[0], 1, 28, 28)
X = X.astype(np.float32)

y = df[['labels']].values.reshape((-1,))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [5]:
train = data_utils.TensorDataset(X_train,y_train)
train_loader = data.DataLoader(train, batch_size=64, shuffle=True)
test = data_utils.TensorDataset(X_test,y_test)
test_loader = data.DataLoader(test, batch_size=64, shuffle=True)
print(len(train), len(test))

23920 5980


In [6]:
from notebooks.model import Net

net = Net()
net.to(device)
net = net.float()


sum(p.numel() for p in net.parameters() if p.requires_grad)

207414

In [7]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy on test images: {100 * correct / total} %')

print('Finished Training')

[1,   200] loss: 0.578
Accuracy on test images: 97.09030100334448 %
[2,   200] loss: 0.107
Accuracy on test images: 97.6923076923077 %
[3,   200] loss: 0.073
Accuracy on test images: 98.12709030100335 %
[4,   200] loss: 0.059
Accuracy on test images: 98.12709030100335 %
[5,   200] loss: 0.051
Accuracy on test images: 98.31103678929766 %
[6,   200] loss: 0.046
Accuracy on test images: 98.29431438127091 %
[7,   200] loss: 0.040
Accuracy on test images: 98.51170568561874 %
[8,   200] loss: 0.037
Accuracy on test images: 98.74581939799332 %
[9,   200] loss: 0.031
Accuracy on test images: 98.7123745819398 %
[10,   200] loss: 0.030
Accuracy on test images: 98.67892976588628 %
Finished Training


In [9]:
torch.save(net.state_dict(), './TMNIST_model.pt')